In [ ]:
!pip install flask flask-cors pyngrok google-genai requests -q

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

import threading, requests
import base64
import os

from google import genai  # usar el nuevo SDK

# Configura tu API Key aquí
client = genai.Client(api_key="KEY")
DEEPGRAM_API_KEY = "KEY"

app = Flask(__name__)
CORS(app)

OPTOMETRY_KEYWORDS = ["mmHg", "x", "esfera", "cilindro", "eje", "dioptrias", "agudeza"]

@app.route("/transcribe", methods=["POST"])
def transcribe():
    try:
        datos = request.get_json()
        print(f"📥 Datos recibidos: {datos}")

        base_url = "https://api.deepgram.com/v1/listen"
        headers = {"Authorization": f"Token {DEEPGRAM_API_KEY}"}
        params = {
            "model": "nova-2-general",  # ¡Cambio: modelo médico para precisión en salud/optometría!
            "language": "es",
            "smart_format": "true",
            "punctuate": "true",  # Puntuación para guiones y símbolos
            "numerals": "true",   # ¡Clave: números como dígitos!
        }

        if "audio_url" in datos:
            headers["Content-Type"] = "application/json"
            data = {"url": datos["audio_url"]}
            response = requests.post(base_url, headers=headers, params=params, json=data)
        elif "audio_base64" in datos:
            audio_bytes = base64.b64decode(datos["audio_base64"])
            mimetype = datos.get("mimetype", "audio/wav")
            print(f"🔊 Mimetype: {mimetype}, tamaño: {len(audio_bytes)} bytes")

            headers["Content-Type"] = mimetype
            response = requests.post(base_url, headers=headers, params=params, data=audio_bytes)
        else:
            return jsonify({"status": "error", "message": "Falta audio_url o audio_base64"}), 400

        print(f"🌐 Deepgram status: {response.status_code}")
        if response.status_code != 200:
            print(f"❌ Deepgram error: {response.text}")
            return jsonify({"status": "error", "message": f"Deepgram: {response.text}"}), response.status_code

        result = response.json()
        print(f"📄 Respuesta completa: {result}")  # Log para ver hipótesis

        if "results" in result and result["results"]["channels"]:
            # Toma la mejor alternativa (con numerals y keywords boost)
            texto = result["results"]["channels"][0]["alternatives"][0]["transcript"]
            print(f"✅ Transcripción: '{texto}'")

            # Si quieres alternativas (para debug), imprime todas
            if "alternatives" in result["results"]["channels"][0]:
                alts = [alt["transcript"] for alt in result["results"]["channels"][0]["alternatives"][1:3]]  # Top 3
                print(f"🔍 Alternativas: {alts}")
        else:
            texto = ""
            print("⚠️ Sin canales en respuesta")

        return jsonify({
            "status": "ok",
            "transcripcion": texto
        })

    except Exception as e:
        print(f"💥 Error: {str(e)}")
        return jsonify({"status": "error", "message": str(e)}), 500

@app.route("/predict", methods=["POST"])
def predict():
    datos = request.get_json()
    sexo = datos.get("sexo", "N/A")
    edad = datos.get("edad", "N/A")
    sintomas = datos.get("sintomas", "N/A")

    prompt = (
        f"Paciente {sexo} de {edad} años con los síntomas: {sintomas}. "
        "Proporciona un diagnóstico presuntivo breve y claro. "
        "No des recetas ni tratamientos, solo posibles condiciones médicas."
    )

    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash",  # o el modelo que tengas acceso
            contents=prompt
        )
        diagnostico = response.text
        status = "ok"
    except Exception as e:
        diagnostico = f"Error: {str(e)}"
        status = "error"

    return jsonify({"prompt": prompt, "diagnostico": diagnostico, "status": status})

def run_app():
    app.run(port=8000)

threading.Thread(target=run_app).start()

!ngrok config add-authtoken KEY

public_url = ngrok.connect(8000).public_url
print("🌍 API pública disponible en:", public_url)

# Prueba rápida
test_data = {"sexo": "masculino", "edad": 45, "sintomas": "dolor de cabeza y mareo"}
resp = requests.post(f"https://chloritic-margarette-unhorizontal.ngrok-free.dev/predict", json=test_data)
print("\n✅ Respuesta de prueba:", resp.json())
